In [1]:
#import necessary libraries
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM, SimpleRNN

In [3]:
import numpy as np
a = open("data_shuffle_input.csv")
net_data_input = np.genfromtxt(a, delimiter=",")
b = open("data_shuffle_target.csv")
net_data_target = np.genfromtxt(b, delimiter=",")
c = open("mydata_shuffle_input.csv")
net_mydata_input = np.genfromtxt(c, delimiter=",")
d = open("mydata_shuffle_target.csv")
net_mydata_target = np.genfromtxt(d, delimiter=",")

In [4]:
import random
from sklearn.model_selection import cross_val_score
net_data_input = net_data_input / 255
net_data_input = net_data_input.reshape(net_data_input.shape[0],1,784)
net_mydata_input = net_mydata_input.reshape(net_mydata_input.shape[0],1,784)

print(net_data_input[0][0])


[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [5]:
acc_per_fold = []
loss_per_fold = []


In [6]:
num = []
for i in range(len(net_data_target)):
    if net_data_target[i][0] == 1:
        num.append(0)
    if net_data_target[i][1] == 1:
        num.append(1)
    if net_data_target[i][2] == 1:
        num.append(2)
    if net_data_target[i][3] == 1:
        num.append(3)
    if net_data_target[i][4] == 1:
        num.append(4)
    if net_data_target[i][5] == 1:
        num.append(5)
    if net_data_target[i][6] == 1:
        num.append(6)
    if net_data_target[i][7] == 1:
        num.append(7)
    if net_data_target[i][8] == 1:
        num.append(8)
    if net_data_target[i][9] == 1:
        num.append(9)
num = np.asarray(num)        
print(type(num))


<class 'numpy.ndarray'>


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5)
acc_per_fold = []
acc = []
loss = []
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(net_data_input, net_data_target, net_mydata_input):


    #make train list input
    joined_list_train = []
    joined_list_test = []

    for i in range(800):
        b = [*net_data_input[train][i][0], *net_mydata_input[train][i][0]]
        joined_list_train.append(b)
        
    for j in range(200):
        a = [*net_data_input[test][j][0], *net_mydata_input[test][j][0]]
        joined_list_test.append(a)

    joined_list_train = np.asarray(joined_list_train)
    joined_list_test = np.asarray(joined_list_test)
    
    
    joined_list_train = joined_list_train.reshape(joined_list_train.shape[0],1,1568)

    joined_list_test = joined_list_test.reshape(joined_list_test.shape[0],1,1568)
    
  # Define the model architecture for net data

    rnn_model_outputs = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_outputs.add(LSTM(512,input_shape=joined_list_train[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_outputs.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_outputs.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} output input')

  # Fit data to model
    history = rnn_model_outputs.fit(joined_list_train, net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    scores = rnn_model_outputs.evaluate(joined_list_test, net_data_target[test], verbose=0)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    predictions = rnn_model_outputs.predict(joined_list_test)
    classes = np.argmax(predictions, axis = 1)
    print(num[test])
    acc_per_fold.append(accuracy_score(num[test],classes))
    
    print('accuracy is ' + str(accuracy_score(num[test],classes)))
    
  # Generate generalization metrics
    acc.append(history.history['acc'])
    loss.append(history.history['loss'])
    

  # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 output input
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 1ms/sample - loss: 1.7989 - acc: 0.4750
Epoch 2/50
800/800 [==============================] - 0s 450us/sample - loss: 0.3720 - acc: 0.9650
Epoch 3/50
800/800 [==============================] - 0s 409us/sample - loss: 0.1016 - acc: 0.9987
Epoch 4/50
800/800 [==============================] - 0s 422us/sample - loss: 0.0365 - acc: 1.0000
Epoch 5/50
800/800 [==============================] - 0s 434us/sample - loss: 0.0182 - acc: 1.0000
Epoch 6/50
800/800 [==============================] - 0s 452us/sample - loss: 0.0113 - acc: 1.0000
Epoch 7/50
800/800 [==============================] - 0s 421us/sample - loss: 0.0079 - acc: 1.0000
Epoch 8/50
800/800 [==============================] - 0s 472us/sample - loss: 0.0061 - acc: 1.0000
Epoch 9/50
800/800 [==============================] - 0s 470us/sample - loss: 0.005

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[1 3 9 2 6 9 5 4 8 7 1 5 2 6 7 1 5 6 8 1 1 5 9 4 9 4 6 6 9 8 7 0 3 7 3 4 7
 6 4 5 4 9 4 4 9 0 3 2 4 8 5 6 7 7 6 1 1 2 4 0 9 2 8 5 9 7 3 8 4 2 3 2 5 8
 4 5 4 8 8 5 9 0 4 2 9 8 0 2 5 3 6 1 2 6 5 9 1 6 0 6 4 1 8 4 6 9 7 0 6 4 7
 7 1 5 4 9 9 9 0 2 7 5 2 3 6 3 7 2 0 0 6 7 4 4 6 0 0 6 3 7 1 9 2 1 3 0 9 0
 7 4 6 5 9 0 9 3 0 1 7 8 8 7 4 4 1 1 3 1 2 9 6 4 5 2 1 7 8 2 5 4 1 1 6 5 8
 5 1 1 2 0 7 4 2 6 0 2 3 9 3 3]
accuracy is 0.8
------------------------------------------------------------------------
Training for fold 2 output input
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 697us/sample - loss: 1.8332 - acc: 0.4500
Epoch 2/50
800/800 [==============================] - 0s 390us/sample - loss: 0.3830 - acc: 0.9638
Epoch 3/50
800/800 [==============================] - 0s 384us/sample - loss: 0.1038 - acc: 0.9962
Epoch 4/50
800/800 [==============================] - 0s 372us/sample - loss: 0.0385 - acc: 1.0000
Epoch 5/50
800/800 [==============================] - 

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[8 4 1 8 2 9 3 3 3 9 9 1 4 3 6 5 3 7 7 0 0 4 5 5 3 3 1 0 5 3 2 8 0 8 6 4 9
 4 6 6 4 5 9 9 0 9 1 7 7 5 4 8 7 3 0 1 9 1 6 4 0 9 3 6 0 6 6 1 6 5 3 1 9 2
 2 2 5 3 8 7 7 5 2 2 5 3 2 4 3 7 8 8 7 4 7 9 0 1 3 4 5 0 4 6 7 3 0 7 9 8 1
 9 5 3 5 5 3 3 4 8 0 9 2 0 4 7 6 8 9 0 4 6 3 9 6 2 1 3 4 8 3 6 2 1 7 0 9 5
 6 2 3 3 5 3 2 3 4 0 9 2 7 9 5 7 9 4 6 9 1 7 0 3 4 7 5 3 4 9 7 4 5 6 1 5 3
 4 5 5 7 3 1 2 6 2 6 2 0 9 3 8]
accuracy is 0.855
------------------------------------------------------------------------
Training for fold 3 output input
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 716us/sample - loss: 1.8610 - acc: 0.4112
Epoch 2/50
800/800 [==============================] - 0s 387us/sample - loss: 0.3901 - acc: 0.9750
Epoch 3/50
800/800 [==============================] - 0s 381us/sample - loss: 0.1086 - acc: 0.9975
Epoch 4/50
800/800 [==============================] - 0s 384us/sample - loss: 0.0412 - acc: 1.0000
Epoch 5/50
800/800 [==============================] 

...

[9 4 1 4 5 7 9 6 2 5 6 2 7 4 3 0 5 0 4 9 7 7 3 7 5 6 4 1 2 5 9 5 0 2 3 7 8
 4 2 2 0 0 4 8 8 8 5 9 5 2 9 9 5 4 5 9 0 7 1 0 4 6 7 8 8 2 7 7 7 3 7 6 3 4
 1 9 7 0 0 2 2 2 2 0 3 9 1 5 5 0 0 2 6 2 2 0 6 5 4 8 8 3 8 0 3 5 8 4 3 1 6
 9 7 7 5 1 4 8 7 8 0 2 7 9 5 1 8 8 1 4 2 6 1 9 9 8 1 7 9 4 5 5 4 4 9 6 2 0
 1 8 2 1 1 7 8 6 6 0 9 7 6 8 1 3 5 7 4 0 8 6 0 6 3 7 9 8 7 9 4 6 6 2 6 1 9
 2 3 3 7 5 1 9 5 4 2 8 2 6 3 8]
accuracy is 0.845
------------------------------------------------------------------------
Training for fold 4 output input
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 686us/sample - loss: 1.8323 - acc: 0.4313
Epoch 2/50
800/800 [==============================] - 0s 379us/sample - loss: 0.3925 - acc: 0.9675
Epoch 3/50
800/800 [==============================] - 0s 390us/sample - loss: 0.1103 - acc: 0.9962
Epoch 4/50
800/800 [==============================] - 0s 388us/sample - loss: 0.0412 - acc: 1.0000
Epoch 5/50
800/800 [==============================] 

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[1 0 2 3 7 2 6 0 2 8 5 1 7 8 6 3 1 3 1 3 5 5 0 8 7 5 9 9 2 5 6 5 7 6 3 8 6
 7 2 0 0 1 7 2 5 3 7 1 1 1 3 9 9 5 2 6 6 8 4 5 3 7 0 0 1 7 9 1 3 6 4 6 6 3
 5 4 2 9 2 9 6 3 2 7 6 0 1 2 5 0 0 7 6 2 8 5 3 8 8 8 4 4 4 6 0 8 8 9 9 3 4
 8 7 9 0 4 1 1 4 8 8 3 4 4 3 6 7 7 3 6 3 9 1 1 7 6 7 0 0 3 8 3 0 5 1 1 2 1
 8 3 6 7 2 1 7 5 0 7 9 7 1 0 8 2 3 6 3 6 4 2 6 6 1 9 3 6 9 4 7 4 2 4 0 8 5
 8 9 8 1 1 1 9 6 4 0 6 8 8 7 8]
accuracy is 0.83
------------------------------------------------------------------------
Training for fold 5 output input
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 676us/sample - loss: 1.8355 - acc: 0.4363
Epoch 2/50
800/800 [==============================] - 0s 370us/sample - loss: 0.3862 - acc: 0.9737
Epoch 3/50
800/800 [==============================] - 0s 375us/sample - loss: 0.1070 - acc: 1.0000
Epoch 4/50
800/800 [==============================] - 0s 373us/sample - loss: 0.0397 - acc: 1.0000
Epoch 5/50
800/800 [==============================] -

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[6 6 6 1 8 0 6 9 9 2 0 3 6 7 1 3 3 5 8 0 5 7 2 3 9 0 5 7 2 5 2 3 8 5 8 0 8
 9 8 7 9 8 7 5 0 5 1 7 3 1 6 3 6 5 6 9 2 5 1 3 6 2 9 1 4 8 7 8 0 5 2 7 4 0
 2 9 0 2 2 7 5 8 8 2 2 9 6 9 4 2 0 0 5 7 3 9 1 8 5 1 1 9 0 4 2 0 1 4 0 1 2
 2 8 0 0 1 6 5 4 4 4 8 1 5 8 8 1 4 8 8 3 9 0 7 4 8 4 2 2 5 8 1 3 9 1 0 3 0
 8 3 4 9 2 4 0 6 8 0 2 9 9 5 7 1 7 2 3 5 7 5 1 8 5 0 1 2 1 5 1 4 8 4 0 2 9
 0 0 1 6 0 6 8 5 8 3 4 8 6 3 5]
accuracy is 0.845


In [8]:

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.4942213678359985 - Accuracy: 0.8%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.5424465751647949 - Accuracy: 0.855%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.5035148239135743 - Accuracy: 0.845%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.5010349369049072 - Accuracy: 0.83%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.6010563659667969 - Accuracy: 0.845%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 0.835 (+- 0.019235384061671322)
> Loss: 0.5284548139572143
------------------------------------------------------------------------


In [9]:
np.savetxt('inputs_cross_validation_acc.csv',acc, delimiter=',')

In [10]:
np.savetxt('inputs_cross_validation_loss.csv',loss, delimiter=',')